Uncertainty of the produced sea ice age variable is computed from the uncertainties of the ice age fractions ($C_i$) multiplied by age ($A$) of the fraction: 

$\sigma_{SIA}^2 = \sum{ A_i * \sigma_{C_{iY}}^2 }, i=1..6$,

where $i$ is the index of the sea ice age fraction: $i=1$ for one-year old ice (i.e., $C_{1Y}$), $i=2$ for two-year old ice, etc.

Each ice age fraction is computed from values of MYI advected for various time (see Eq. NNN). Therefore, the uncertainty of each fraction is computed from uncertaimties of advected MYI fields:

$\sigma_{C_{iY}}^2 = \sum{ \sigma_{C_{A,j}}^2 }, j = 1..6$,

where $j$ is the index of the advected MYI field, with $j=1$ being the yongest advected MYI field.

Uncertainty of the FYI is computed using the uncertainty from the satellite SIC observations and the youngest advected MYI field:

$\sigma_{FYI}^2 = \sigma_{C_{OBS}}^2 + \sigma_{C_{A,1}}^2$

Uncertaimty of the advected MYI field is computed from uncertainties of the satellite derived SIC and SID:

$\sigma_{C_{A,j}}^2 = \sigma_{SIC,j}^2 + \sigma_{SID,j}^2$

Both SIC and SID uncertaimties are iteratively accumulated during the advection of the MYI fields. 

The concentration in the advected field is capped by the observed concentration. In other words it is a minimum values of the advected and the observed concentration (see Eq. NNN). Therefore, the uncertainty of the advected field is the uncertainty of the minimal concentration:

$\sigma_{SIC,j,n}^2 = $

$ k_n \left< \sigma_{SIC,j,n-1}^2 \right> \text{, if } SIC_{n-1} < C_{OBS,n}$

$\sigma_{C_{OBS,n}}^2 \text{, otherwise}$

where $n$ denotes the current step, $n-1$ - the previous step, $\left< \right>$ - operation of advection, $k$ - factor for change of area (and, therefore, uncertainty) due to divergence/convergence (see Eq. NNN).


The uncertainty associated with the drift is computed the following way. First, the uncertainty of the smoothed ice drift product ($\sigma_{S}$) is computed using the provided uncertainties of the OSI-SAF SID product ($\sigma_D$):

$\sigma_{S}^2 = \frac{\sum_k^N{\sigma_{D_k}}}{N}$,

where $\sigma_{D_k}$ is the uncertaimty of gridded ice drift in a $k$-th neighbour and $N$ is number of neighbours.

Next, the integrated uncertainty of the ice drift ($\sigma_{I}$) is iteratively acumulated with advection:

$\sigma_{I,n}^2 = \left< \sigma_{I,n-1}^2 \right> + \sigma_{S}^2$

After some steps of advection, the integrated uncertainty of sea ice drift defines a radius of a circle, where the advected mesh element could have drifted. This circle may include other advected elements with their respective concentrations. Therefore the uncertainty of advected MYI concentration, associated with the uncertaimty of ice drift $\sigma_{SID,j}$ is computed as a standard deviation of concentrations in this circle:

$\sigma_{SID,j}^2 = (\sum_m^M{ C_{A,j,m} - \hat{C}_{A,j})}^2/M$

where $m$ is the index of the elements in the circle, $\hat{C}_{A,j}$ average concentration in the circle, M – number of elements in the circle.


In [ ]:
import glob
from datetime import datetime
import os
from multiprocessing import Pool

import numpy as np

from lmsiage.uncertainty import ComputeSicUncertainty
from lmsiage.mesh_file import MeshFile
from lmsiage.zarr_index_tools import cleanup_missing_files, files_with_array
cleanup_missing_files()

In [ ]:
n_steps = 2210  # should be 2210 to cover 6 years

sia_dir = 'NERSC_arctic25km_sea_ice_age_v2p1/zarr'
mesh_dir = f'{sia_dir}/mesh'
unc_dir = f'{sia_dir}/uncert'
age_dir = f'{sia_dir}/age'
sid_dir = 'OSISAF_ice_drift_CDR_postproc'
sid_files = sorted(glob.glob(f'{sid_dir}/*/ice_drift*npz'))
sid_dates = [datetime.strptime(os.path.basename(sid_file).split('-')[-1].split('.')[0], '%Y%m%d%H%M%S')
          for sid_file in sid_files]
print(f'Found {len(sid_files)} ice drift files from {sid_dates[0]} to {sid_dates[-1]}')

mesh_init_file = 'mesh_arctic_ease_25km_max7.npz'
xc = np.load(mesh_init_file)['xc']
yc = np.load(mesh_init_file)['yc']

In [ ]:
csu = ComputeSicUncertainty(sid_files, sid_dates, mesh_dir, age_dir, unc_dir, n_steps, xc, yc)
for year in range(1991, 1995):
    csu.compute_obs_uncertainty(year=year)

In [ ]:
unc_sic_files = sorted(files_with_array('unc_sic'))
unc_sic_dates = [datetime.strptime(f.split('/')[-1], 'unc_%Y%m%d.zip') for f in unc_sic_files]
start_indices = [i for i in range(0, len(unc_sic_dates)) if unc_sic_dates[i].month == 9 and unc_sic_dates[i].day == 15]

In [ ]:
for start_idx in start_indices:
    csu.compute_min_uncertainty(unc_sic_dates, start_idx)

In [ ]:
print(MeshFile('NERSC_arctic25km_sea_ice_age_v2p1/zarr/uncert/1994/unc_19941231.zip').read_names())

import matplotlib.pyplot as plt
unc_sic = MeshFile('NERSC_arctic25km_sea_ice_age_v2p1/zarr/uncert/1994/unc_19941231.zip').load(['unc_sic1991'], as_dict=False)[0]
x, y, t = MeshFile('NERSC_arctic25km_sea_ice_age_v2p1/zarr/mesh/1994/mesh_19941231.zip').load(['x', 'y', 't'], as_dict=False)
plt.tripcolor(x, y, t, unc_sic)
plt.colorbar()